In [301]:
import os
from os import environ

import requests
import io
import string

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

import ipywidgets as widgets
from IPython.display import display
from IPython.html.widgets import interactive

In [302]:
cluster_variables=['Route','Flight Number','Weekday']

In [303]:
Raw=pd.read_csv('JAN2017_APR2019_P.csv',sep=';')

Raw=Raw.loc[Raw['Sched Airport Depart'].apply(lambda x: (x=='CPH')|(x=='LCY')|(x=='LUX')|(x=='MAD')|(x=='MXP')|(x=='NCE')|(x=='VIE'))&\
Raw['Sched Airport Arriva'].apply(lambda x: (x=='CPH')|(x=='LCY')|(x=='LUX')|(x=='MAD')|(x=='MXP')|(x=='NCE')|(x=='VIE')),:]
Raw['Route']=Raw['Sched Airport Depart']+'-'+Raw['Sched Airport Arriva']
Raw['Route']='LUX-'+Raw['Route'].apply(lambda x: x.replace('LUX','').replace('-',''))

Raw=Raw.loc[Raw['Load Factor'].apply(lambda x: x.replace(',','').isnumeric()),:]

Raw=Raw.rename(columns={'Coupons+YQ in current bookings': 'Revenue'})

Raw['Revenue']=Raw['Revenue'].apply(lambda x: np.float(x.replace('EUR','').replace(',','.').replace(' ','')))

Raw['Pax total']=Raw['Pax total'].apply(lambda x: np.float(x.replace('PAX','').replace(',','.').replace(' ','')))

Raw=Raw.rename(columns={'Revenue': 'Revenue_actual', 'Pax total': 'Pax_actual'})

Raw['Calendar day']=Raw['Calendar day'].apply(lambda x: pd.to_datetime(x.split('/')[2]+'-'+x.split('/')[1]+'-'+x.split('/')[0]))
Raw['LF_actual']=Raw['Load Factor'].apply(lambda x: float(x.replace(',','.').replace('%',''))/100)
Raw['LF_actual']=Raw['Load Factor'].apply(lambda x: float(x.replace(',','.').replace('%','')))
Raw['Direction']=Raw[['Sched Airport Depart','Sched Airport Arriva']].apply(lambda x: 'O' if x[0]=='LUX' and (x[1]=='CPH' or x[1]=='LCY') else 'I',axis=1)

Raw=Raw.rename(columns={'Calendar day': 'Departure Date'})
Raw=Raw.loc[Raw['Departure Date']>=pd.to_datetime('2018-04-01'),:]

Raw['Yield_actual']=Raw['Revenue_actual']/Raw['Pax_actual']

### Seasonality Clustering

In [304]:
Raw['SEASON']='M'

In [305]:
# LUX-CPH

    ## LOW

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-07-31'))&(Raw['Departure Date']<=pd.to_datetime('2017-08-11')),'SEASON']='L'    

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-07-31'))&(Raw['Departure Date']<=pd.to_datetime('2017-08-11')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-08-14'))&(Raw['Departure Date']<=pd.to_datetime('2017-09-17')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-19'))&(Raw['Departure Date']<=pd.to_datetime('2017-12-22')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-27'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-07')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-31'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-17')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-27'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-06')),'SEASON']='L'

    ## HIGH

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-04-05'))&(Raw['Departure Date']<=pd.to_datetime('2017-04-08')),'SEASON']='H'    

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-04-28'))&(Raw['Departure Date']<=pd.to_datetime('2017-04-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-05-23'))&(Raw['Departure Date']<=pd.to_datetime('2017-05-25')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-06-01'))&(Raw['Departure Date']<=pd.to_datetime('2017-06-05')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-06-22'))&(Raw['Departure Date']<=pd.to_datetime('2017-06-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-07-13'))&(Raw['Departure Date']<=pd.to_datetime('2017-07-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-08-12'))&(Raw['Departure Date']<=pd.to_datetime('2017-08-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-10-27'))&(Raw['Departure Date']<=pd.to_datetime('2017-10-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-15'))&(Raw['Departure Date']<=pd.to_datetime('2017-12-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-11')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']==pd.to_datetime('2018-04-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-27'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-10')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-21')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-11'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-12')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-28')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-20'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-15'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-05'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-29'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-30')),'SEASON']='H'

In [306]:
# CPH-LUX

    ## HIGH

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2017-01-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-01-06'))&(Raw['Departure Date']<=pd.to_datetime('2017-01-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-11-04'))&(Raw['Departure Date']<=pd.to_datetime('2017-11-06')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-03')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-15'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-07')),'SEASON']='H'

In [307]:
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY'),'SEASON']='H'
Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX'),'SEASON']='H'

In [308]:
# LUX-LCY

    ## LOW

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']==pd.to_datetime('2018-01-07')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-13')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-18')),'SEASON']='L' 
   
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-03')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-07'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-10')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-15')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-01')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-13')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-21')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-27')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-04')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-22'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-06')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-06'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-07')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-13'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-22')),'SEASON']='L'

In [309]:
# LCY-LUX

    ## LOW

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-07')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-13')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-18')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-15')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-05-01')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-13')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-21')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-27')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-04')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-22'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-06')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-06'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-22')),'SEASON']='L'

In [310]:
# LUX-MAD

    ## LOW  

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-07')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-28')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']==pd.to_datetime('2018-05-11')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']==pd.to_datetime('2018-11-02')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-13')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-19'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-28')),'SEASON']='L'

    ## HIGH
    
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-22'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']==pd.to_datetime('2018-03-30')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-20')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-27')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']==pd.to_datetime('2018-05-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-25'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-26')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']==pd.to_datetime('2018-12-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']==pd.to_datetime('2018-12-14')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-14'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-11'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-12')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MAD')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-18'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-19')),'SEASON']='H'

In [311]:
# MAD-LUX

    ## LOW
    
Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-15')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-02-17')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-20'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-02')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-03'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-04')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-06'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-29')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-12-31')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-08'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-21')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-25'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-28')),'SEASON']='L'

    ## HIGH

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-01-01')),'SEASON']='H'    
    
Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-06'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-04-02')),'SEASON']='H' 

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-04-08')),'SEASON']='H' 

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-04-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-04-22')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-05-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-05-14')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-05-21')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-05-27')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-06-03')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-06-10')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-06-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-06-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-07-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-07-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-07-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-07-22')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-07-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-08-31')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-09-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-09-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-09-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-09-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-09-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-09-30')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-10-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-10-14')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-10-21')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-10-28')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-11-04')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-11-05')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-11-06')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-11-11')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-11-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-22'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2019-04-14')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MAD')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-21'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-22')),'SEASON']='H'

In [312]:
# LUX-MXP

    ## LOW
    
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-07')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']==pd.to_datetime('2019-11-02')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-06')),'SEASON']='L'

    ## HIGH

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']==pd.to_datetime('2018-02-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-21'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-15'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-21'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-12'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-20')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-27')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-03'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-04')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-25')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-31'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-07'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-21'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-22')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-05'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-06')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-12'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-23'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-30'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-31')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-06'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-14')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-20'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-21')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-27'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-28')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-04'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-05')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-12')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-19')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-26')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-31'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-31')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-21'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-26')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-30')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-07'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-01'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-07'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-14'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-28'))&(Raw['Departure Date']<=pd.to_datetime('2019-03-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-07'))&(Raw['Departure Date']<=pd.to_datetime('2019-03-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-14'))&(Raw['Departure Date']<=pd.to_datetime('2019-03-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-21'))&(Raw['Departure Date']<=pd.to_datetime('2019-03-22')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-28'))&(Raw['Departure Date']<=pd.to_datetime('2019-03-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-04'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-05')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='MXP')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-18'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-18')),'SEASON']='H'

In [313]:
# MXP-LUX

    ## LOW
    
Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-04')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-06'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-09')),'SEASON']='L'

    ## HIGH 
    
Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-07'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-05'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-05')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-19')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-26')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-05'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-05')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-12'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-12')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-19')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-26')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-02'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-16'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-23'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-30'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-30')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-07'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-14')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-21'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-22')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-28')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-04'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-04')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-11'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-11')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-25'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-25')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-02'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-09')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-16'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-23'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-10')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-15'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-22'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-22')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-29'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-05'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-05')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-12'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-12')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-19')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-26')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-03'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-03')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-10')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-05'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-06')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-04'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-04')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-11'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-11')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-25'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-25')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-04'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-04')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-11'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-11')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-25'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-25')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='MXP')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-25'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-25')),'SEASON']='H'

In [314]:
# LUX-NCE

    ## LOW
    
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-06')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-23')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-27'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-08')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-28')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-03'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-05')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-11')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-15'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-19')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-26')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-30'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-06')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-16')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-21')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-05')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-23'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-16'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-29')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-04'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-06')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-20')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-28')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-02'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-21')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-30'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-31')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-03'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-19')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-12')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-19'))&(Raw['Departure Date']<=pd.to_datetime('2019-03-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-15'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-04')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-10'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-12')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-14'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-16')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-22'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-28')),'SEASON']='L'

    ## HIGH

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-11')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-11'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-30'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-31')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-27'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-10')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-20')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-25')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-22'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-12'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-28')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-21'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-15'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-11'))&(Raw['Departure Date']<=pd.to_datetime('2019-03-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='NCE')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-05'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-06')),'SEASON']='H'

In [315]:
# NCE-LUX

    ## LOW
    
Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-03'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-20'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-12')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-05')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-12')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-30')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-03'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-03')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-15'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-18')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-29'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-08')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-11'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-15')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-23')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-29')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-02'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-05')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-13')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-16'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-20')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-23'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-27')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-30'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-03')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-06'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-21'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-31')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-06'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-23')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-15')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-10')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-11')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-16'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-18')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-26')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-30')),'SEASON']='L'

    ## HIGH

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-02')),'SEASON']='H'
    
Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-16'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-03-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-03-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-07'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-12'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-25'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-28')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-06-24'))&(Raw['Departure Date']<=pd.to_datetime('2018-06-25')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-07'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-21'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-22')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-15')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-19')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-30'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-31')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-09-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-04')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-29'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-31')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-06')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-22'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-03-14'))&(Raw['Departure Date']<=pd.to_datetime('2019-03-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='NCE')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-19'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-22')),'SEASON']='H'

In [316]:
Raw=Raw[['Departure Date','Route','Flight Number','Direction','SEASON','LF_actual','Revenue_actual','Pax_actual','Yield_actual']]

#Raw['Year']=Raw['Departure Date'].apply(lambda x: x.year)
Raw['Year']='2017'
Raw['Year']=Raw['Departure Date']\
.apply(lambda x: '2017-2018' if (x>=pd.to_datetime('2017-04-01'))&(x<=pd.to_datetime('2018-03-31'))\
else ('2018-2019' if (x>=pd.to_datetime('2018-04-01'))&(x<=pd.to_datetime('2019-03-31')) else 'out'))
Raw=Raw.loc[Raw['Year'].apply(lambda x: (x=='2017-2018')|(x=='2018-2019')),:]

Raw['Month']=Raw['Departure Date'].apply(lambda x: x.month)
Raw['Weekday']=Raw['Departure Date'].apply(lambda x: x.weekday()+1)

Raw=Raw[[x for x in Raw.columns if 'LF' not in x and 'Revenue' not in x and 'Pax' not in x and 'Yield' not in x]\
        +[x for x in Raw.columns if 'LF' in x or 'Revenue' in x or 'Pax' in x or 'Yield' in x]]

Raw['Departure Date']=Raw['Departure Date'].apply(lambda x: str(x).split(' ')[0])
Raw['Standardized Departure Date']=Raw['Departure Date'].apply(lambda x: '2019-'+x[5:])

In [318]:
Bcg=pd.read_csv('Spoil upside calculation database with new RPK.csv',sep=';',skiprows=0)

Bcg['Route']='LUX'+'-'+Bcg['leg']

Bcg=Bcg.loc[((Bcg['leg']=='CPH')|(Bcg['leg']=='LCY')|(Bcg['leg']=='MAD')|(Bcg['leg']=='MXP')|(Bcg['leg']=='NCE')|(Bcg['leg']=='VIE'))&(Bcg['uplift_population']=='YES'),:]
Bcg.index=range(Bcg.shape[0])

Bcg['Pax_old']=Bcg['Sum_RPK']/Bcg['Stage Length']
Bcg['Pax_new']=Bcg['New RPK']/Bcg['Stage Length']

Bcg=Bcg[['Departure Date','Route','Flight Number','SEASON','Sum_ASK','Sum_RPK','New RPK','Sum_Revenues','New Revenue',
        'Pax_old','Pax_new']]

Bcg['Sum_Revenues']=Bcg['Sum_Revenues'].astype('float')
Bcg['New Revenue']=Bcg['New Revenue'].astype('float')

Bcg['LF_old']=Bcg['Sum_RPK']/Bcg['Sum_ASK']
Bcg['LF_new']=Bcg['New RPK']/Bcg['Sum_ASK']

Bcg['Yield_old']=Bcg['Sum_Revenues']/Bcg['Pax_old']
Bcg['Yield_new']=Bcg['New Revenue']/Bcg['Pax_new']

Bcg=Bcg[[x for x in Bcg.columns if 'ASK' not in x and 'RPK' not in x]]

Bcg['Departure Date']=Bcg['Departure Date'].apply(lambda x: pd.to_datetime(x.split('.')[2]+'-'+x.split('.')[1]+'-'+x.split('.')[0]))

Bcg['Month']=Bcg['Departure Date'].apply(lambda x: x.month)
Bcg['Weekday']=Bcg['Departure Date'].apply(lambda x: x.weekday()+1)

Bcg=Bcg[[x for x in Bcg.columns if 'LF' not in x]+[x for x in Bcg.columns if 'LF' in x]]

Bcg['Departure Date']=Bcg['Departure Date'].apply(lambda x: str(x).split(' ')[0])
Bcg['Standardized Departure Date']=Bcg['Departure Date'].apply(lambda x: '2019-'+x[5:])

Bcg_clustersize=Bcg.groupby(cluster_variables)['Departure Date'].apply(lambda x: len(set(x))).reset_index()

Bcg=Bcg.rename(columns={'Sum_Revenues': 'Revenue_old', 'New Revenue': 'Revenue_new'})

In [319]:
Bcg.shape

(447, 15)

In [320]:
Bcg[['Route','Flight Number','Weekday','SEASON']].sort_values(by=['Route','Flight Number','Weekday','SEASON']).drop_duplicates().shape

(204, 4)

In [321]:
Bcg['Revenue_old'].sum()

1640385.0

In [323]:
Bcg=Bcg.merge(Raw[cluster_variables+['Standardized Departure Date','LF_actual','Revenue_actual','Pax_actual','Yield_actual']],on=cluster_variables,how='left')

Bcg=Bcg.dropna(how='any')

Bcg['Standardized Departure Date_x']=Bcg['Standardized Departure Date_x'].apply(lambda x: pd.to_datetime(x))
Bcg['Standardized Departure Date_y']=Bcg['Standardized Departure Date_y'].apply(lambda x: pd.to_datetime(x))

Bcg['Date_Diff']=(Bcg['Standardized Departure Date_x']-Bcg['Standardized Departure Date_y']).apply(lambda x: abs(x.days))
Bcg['Date_Diff_min']=Bcg.groupby(['Standardized Departure Date_x','Flight Number','Weekday','SEASON'])['Date_Diff'].transform('min')

Bcg=Bcg.loc[(Bcg['Date_Diff']==Bcg['Date_Diff_min'])&(Bcg['Date_Diff']<=7),:]

BcgRaw_clustersize=Bcg.groupby(cluster_variables)['Departure Date'].apply(lambda x: len(set(x))).reset_index()

In [324]:
Bcg['Revenue_old'].sum()

1637172.0

In [325]:
Bcg['Revenue_actual'].sum()

2331406.3499999996

In [326]:
Missing=Bcg_clustersize.merge(BcgRaw_clustersize,on=cluster_variables,how='left')
Missing.loc[pd.isnull(Missing['Departure Date_y']),:]

,Route,Flight Number,Weekday,Departure Date_x,Departure Date_y


In [327]:
Bcg[['Route','Flight Number','Weekday','SEASON']].sort_values(by=['Route','Flight Number','Weekday','SEASON']).drop_duplicates().shape

(204, 4)

In [330]:
Bcg=Bcg.sort_values(by=cluster_variables+['SEASON'])

Box_LF_old=Bcg[cluster_variables+['SEASON']+['LF_old']].rename(columns={'LF_old': 'LF'})
Box_LF_old['Sample']='Actuals 2017-2018'
Box_LF_actual=Bcg[cluster_variables+['SEASON']+['LF_actual']].rename(columns={'LF_actual': 'LF'})
Box_LF_actual['Sample']='Actuals 2018-2019'
Box_LF_new=Bcg[cluster_variables+['SEASON']+['LF_new']].rename(columns={'LF_new': 'LF'})
Box_LF_new['Sample']='Model Bcg'
Box_LF=pd.concat([Box_LF_old,Box_LF_new,Box_LF_actual])
Box_LF.to_csv('GUI_in_LF.csv',index=False)

Box_Revenue_old=Bcg[cluster_variables+['SEASON']+['Revenue_old']].rename(columns={'Revenue_old': 'Revenue'})
Box_Revenue_old['Sample']='Actuals 2017-2018'
Box_Revenue_actual=Bcg[cluster_variables+['SEASON']+['Revenue_actual']].rename(columns={'Revenue_actual': 'Revenue'})
Box_Revenue_actual['Sample']='Actuals 2018-2019'
Box_Revenue_new=Bcg[cluster_variables+['SEASON']+['Revenue_new']].rename(columns={'Revenue_new': 'Revenue'})
Box_Revenue_new['Sample']='Model Bcg'
Box_Revenue=pd.concat([Box_Revenue_old,Box_Revenue_new,Box_Revenue_actual])
Box_Revenue.to_csv('GUI_in_Revenue.csv',index=False)

Box_Pax_old=Bcg[cluster_variables+['SEASON']+['Pax_old']].rename(columns={'Pax_old': 'Pax'})
Box_Pax_old['Sample']='Actuals 2017-2018'
Box_Pax_actual=Bcg[cluster_variables+['SEASON']+['Pax_actual']].rename(columns={'Pax_actual': 'Pax'})
Box_Pax_actual['Sample']='Actuals 2018-2019'
Box_Pax_new=Bcg[cluster_variables+['SEASON']+['Pax_new']].rename(columns={'Pax_new': 'Pax'})
Box_Pax_new['Sample']='Model Bcg'
Box_Pax=pd.concat([Box_Pax_old,Box_Pax_new,Box_Pax_actual])
Box_Pax.to_csv('GUI_in_Pax.csv',index=False)

Box_Yield_old=Bcg[cluster_variables+['SEASON']+['Yield_old']].rename(columns={'Yield_old': 'Yield'})
Box_Yield_old['Sample']='Actuals 2017-2018'
Box_Yield_actual=Bcg[cluster_variables+['SEASON']+['Yield_actual']].rename(columns={'Yield_actual': 'Yield'})
Box_Yield_actual['Sample']='Actuals 2018-2019'
Box_Yield_new=Bcg[cluster_variables+['SEASON']+['Yield_new']].rename(columns={'Yield_new': 'Yield'})
Box_Yield_new['Sample']='Model Bcg'
Box_Yield=pd.concat([Box_Yield_old,Box_Yield_new,Box_Yield_actual])
Box_Yield.to_csv('GUI_in_Yield.csv',index=False)